In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import binomtest,chisquare,ttest_1samp,chi2_contingency,ttest_ind,f_oneway,pearsonr
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,MinMaxScaler,StandardScaler,PolynomialFeatures
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest,f_classif,f_regression
from sklearn.neighbors import LocalOutlierFactor


In [2]:
datasets = sns.get_dataset_names()
datasets

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'dowjones',
 'exercise',
 'flights',
 'fmri',
 'geyser',
 'glue',
 'healthexp',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'seaice',
 'taxis',
 'tips',
 'titanic']

# exercice 1

In [3]:
df = sns.load_dataset("tips")
df.head()
df.dropna(inplace=True)

In [4]:
# ici on a  5 variables discretres : sex	smoker	day	time	size
# on a deux variables continues : 	total_bill	tip

#question 1

df.groupby('total_bill')['size'].describe()

,count,mean,std,min,25%,50%,75%,max
total_bill,,,,,,,,
3.07,1.0,1.0,NaN,1.0,1.00,1.0,1.00,1.0
5.75,1.0,2.0,NaN,2.0,2.00,2.0,2.00,2.0
7.25,2.0,1.5,0.707107,1.0,1.25,1.5,1.75,2.0
7.51,1.0,2.0,NaN,2.0,2.00,2.0,2.00,2.0
7.56,1.0,2.0,NaN,2.0,2.00,2.0,2.00,2.0
...,...,...,...,...,...,...,...,...
45.35,1.0,3.0,NaN,3.0,3.00,3.0,3.00,3.0
48.17,1.0,6.0,NaN,6.0,6.00,6.0,6.00,6.0
48.27,1.0,4.0,NaN,4.0,4.00,4.0,4.00,4.0


In [5]:
df['size'].unique()

array([2, 3, 4, 1, 6, 5])

In [6]:
alpha = 0.02
p_value = ttest_ind(df['total_bill'],df['size']).pvalue

if p_value < alpha:
    print("nous avons suffissament dhypothese pour rejetter H0")
else:
    print("nous n'avons pas suffisament d'hypothese pour rejetter H0 ")

nous avons suffissament dhypothese pour rejetter H0


# 2. Le pourboire moyen (tip) est-il significativement différent pour le déjeuner que pour le dîner ?

In [7]:
df['tip'].head()
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [8]:
df.groupby('time')['tip'].mean()

/tmp/ipykernel_92026/2702322296.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('time')['tip'].mean()


time
Lunch     2.728088
Dinner    3.102670
Name: tip, dtype: float64

In [9]:
Lunch_time = df.query("`time`=='Lunch'")
Dinner_time = df.query("`time`=='Dinner'")

alpha = 0.02

p_value = ttest_ind(Lunch_time['tip'],Dinner_time['tip']).pvalue

if p_value < alpha:
    print("nous avons suffisament d'hypothes pour rejeter H0")
else:
    print("nous n'avons pas suffisament d'hypothese pour rejetter H0")

nous n'avons pas suffisament d'hypothese pour rejetter H0


# 3. Existe-t-il une dépendance entre le jour de la semaine (day) et le fait que les clients  soient fumeurs (smoker) ?

In [10]:
crostab = pd.crosstab(df['day'],df['smoker'])

alpha = 0.02

p_value = chi2_contingency(crostab).pvalue

if p_value < alpha:
    print("nous avons assez dhypothese pour rejetter H0")
else:
    print("nous n'avons pas assez de preuvent pour rejetter H0")

nous avons assez dhypothese pour rejetter H0


# 4. Le pourboire moyen (tip) est-il supérieur à 4,00 $ ?

In [11]:
df['tip'].describe()

count    244.000000
mean       2.998279
std        1.383638
min        1.000000
25%        2.000000
50%        2.900000
75%        3.562500
max       10.000000
Name: tip, dtype: float64

In [12]:
alpha  = 0.02
p_value = ttest_1samp(df['tip'],popmean=4.00,alternative='greater').pvalue

if p_value < alpha :
    print("nous avons suffisament dhypothese pous rejetter H0")
else:
    print("nous navons pas suffisament de preuve pour rejetter H0")

nous navons pas suffisament de preuve pour rejetter H0


# 5. Le pourcentage de pourboire est-il différent selon les quatre jours de la semaine (day) ?

In [13]:
df.groupby('day')['tip'].mean()

/tmp/ipykernel_92026/4148581069.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('day')['tip'].mean()


day
Thur    2.771452
Fri     2.734737
Sat     2.993103
Sun     3.255132
Name: tip, dtype: float64

In [14]:
df.groupby('day')['tip'].apply(list)


/tmp/ipykernel_92026/3132504188.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('day')['tip'].apply(list)


day
Thur    [4.0, 3.0, 2.71, 3.0, 3.4, 1.83, 5.0, 2.03, 5....
Fri     [3.0, 3.5, 1.0, 4.3, 3.25, 4.73, 4.0, 1.5, 3.0...
Sat     [3.35, 4.08, 2.75, 2.23, 7.58, 3.18, 2.34, 2.0...
Sun     [1.01, 1.66, 3.5, 3.31, 3.61, 4.71, 2.0, 3.12,...
Name: tip, dtype: object

In [15]:
alpha = 0.02

p_value = f_oneway(*df.groupby('day')['tip'].apply(list)).pvalue

if p_value < alpha :
    print("nous avons suffisamen dhypothese pous rejetter H0")
else:
    print("nous navons pas suffisament de preuve pour rejetter H0")

nous navons pas suffisament de preuve pour rejetter H0


/tmp/ipykernel_92026/1882088737.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  p_value = f_oneway(*df.groupby('day')['tip'].apply(list)).pvalue


# 6. Si l'on estime que 50% des clients paient en espèces, la proportion observée de femmes payant est-elle statistiquement différente de 50% ?

In [16]:
# Question 6 : Proportion de femmes = 50% ?
# Test binomial

from scipy.stats import binomtest

alpha = 0.05

# Compter le nombre de femmes
k = (df['sex'] == 'Female').sum()  # Nombre de succès (femmes)
n = len(df)                         # Taille de l'échantillon
p_hypothese = 0.5                   # Hypothèse : 50%

print(f"Nombre de femmes : {k}/{n}")
print(f"Proportion observée : {k/n:.2%}")

# Test binomial (bilatéral par défaut)
result = binomtest(k=k, n=n, p=p_hypothese)
p_value = result.pvalue

print(f"P-value : {p_value:.4f}")

if p_value < alpha:
    print("✅ La proportion de femmes est significativement différente de 50%")
else:
    print("❌ Pas de différence significative avec 50%")

Nombre de femmes : 87/244
Proportion observée : 35.66%
P-value : 0.0000
✅ La proportion de femmes est significativement différente de 50%


# 7. Existe-t-il une relation linéaire entre la facture totale (total_bill) et le montant du pourboire (tip) ?

In [17]:
df['total_bill']
df['tip']

0      1.01
1      1.66
2      3.50
3      3.31
4      3.61
       ... 
239    5.92
240    2.00
241    2.00
242    1.75
243    3.00
Name: tip, Length: 244, dtype: float64

In [18]:
alpha = 0.02

p_value = pearsonr(df['total_bill'],df['tip']).pvalue


if p_value < alpha :
    print("nous avons suffisamen dhypothese pous rejetter H0")
else:
    print("nous navons pas suffisament de preuve pour rejetter H0")

nous avons suffisamen dhypothese pous rejetter H0


# 8. La répartition des clients fumeurs vs non-fumeurs est-elle uniforme sur l'ensemble de  l'échantillon ?


In [19]:
df['smoker'].value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [20]:
df['smoker'].value_counts(normalize=True)

smoker
No     0.618852
Yes    0.381148
Name: proportion, dtype: float64

In [21]:
alpha = 0.02
p = 0.5

k = df['smoker'].value_counts()['Yes']
k

np.int64(93)

In [22]:
n = len(df)
n

244

In [23]:
p_value = binomtest(k=k,n=n,p=p).pvalue
if p_value < alpha :
    print("nous avons suffisament dhypothese pous rejetter H0")
else:
    print("nous navons pas suffisament de preuve pour rejetter H0")


nous avons suffisament dhypothese pous rejetter H0


# 9. La moyenne de la facture totale (total_bill) est-elle la même pour les groupes de taille 2, 3 et 4 ?


In [24]:

size_choice = df.query("`size`==2 & `size` == 3 & `size` == 4")



In [25]:

alpha = 0.02

p_value = ttest_ind(size_choice['total_bill'],size_choice['total_bill']).pvalue

if p_value < alpha:
    print("nous avons suffisament d'hypothes pour rejeter H0")
else:
    print("nous n'avons pas suffisament d'hypothese pour rejetter H0")

nous n'avons pas suffisament d'hypothese pour rejetter H0


/home/romaric420/VerificationConnaissance/.venv/lib/python3.12/site-packages/scipy/_lib/deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


In [26]:
# Question 9 : La moyenne de total_bill est-elle la même pour les tailles 2, 3 et 4 ?
# ANOVA (Analysis of Variance) pour comparer 3 groupes

from scipy.stats import f_oneway

alpha = 0.05

# Filtrer les tailles 2, 3, 4 (avec OR, pas AND)
df_filtered = df[df['size'].isin([2, 3, 4])]

# Vérification
print("Moyennes par groupe :")
print(df_filtered.groupby('size')['total_bill'].mean())

# Créer les 3 groupes séparés
group_2 = df_filtered[df_filtered['size'] == 2]['total_bill']
group_3 = df_filtered[df_filtered['size'] == 3]['total_bill']
group_4 = df_filtered[df_filtered['size'] == 4]['total_bill']

# ANOVA à un facteur (one-way ANOVA)
stat, p_value = f_oneway(group_2, group_3, group_4)

print(f"\nStatistique F : {stat:.3f}")
print(f"P-value : {p_value:.4f}")

if p_value < alpha:
    print("✅ Les moyennes sont significativement différentes entre les 3 groupes")
else:
    print("❌ Pas de différence significative entre les groupes")

Moyennes par groupe :
size
2    16.448013
3    23.277632
4    28.613514
Name: total_bill, dtype: float64

Statistique F : 49.793
P-value : 0.0000
✅ Les moyennes sont significativement différentes entre les 3 groupes


In [27]:
df[df['size'].isin([2, 3, 4])].groupby('size')['total_bill'].mean()


size
2    16.448013
3    23.277632
4    28.613514
Name: total_bill, dtype: float64

# 10. La répartition des heures de repas (Déjeuner vs Dîner) est-elle la même que la  répartition des fumeurs (Oui vs Non) ?

In [28]:
df['smoker'].value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [29]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [30]:
crosstab = pd.crosstab(df['time'],df['smoker'])

alpha = 0.02

p_value = chi2_contingency(crosstab).pvalue


if p_value < alpha:
    print("nous avons suffisament d'hypothes pour rejeter H0")
else:
    print("nous n'avons pas suffisament d'hypothese pour rejetter H0")

nous n'avons pas suffisament d'hypothese pour rejetter H0


 # 1Test BILATÉRAL (par défaut) Question : "La valeur est-elle DIFFÉRENTE de X ?"
 # Test UNILATÉRAL DROITE : alternative='greater' Question : "La valeur est-elle SUPÉRIEURE à X ?"
 # Test UNILATÉRAL GAUCHE : alternative='less' Question : "La valeur est-elle INFÉRIEURE à X ?"